## 1️⃣ Setup and Imports

In [ ]:
# Import required libraries
import sys
from pathlib import Path
from IPython.display import display, Markdown, HTML
import json

# Add src to path
sys.path.insert(0, str(Path.cwd()))

from src.models.checkpoint import Checkpoint, GatheredContext
from src.models.state import create_initial_state
from src.modules.context_manager import ContextManager
from src.graph.learning_graph import create_learning_graph

print("✅ All imports successful!")

ModuleNotFoundError: No module named 'src'

## 2️⃣ Test Individual Components

### Test 1: Create a Checkpoint

In [ ]:
# Create a learning checkpoint
checkpoint = Checkpoint(
    topic="Python Functions and Parameters",
    objectives=[
        "Understand function definition syntax",
        "Learn how to use function parameters",
        "Master return values and function calls"
    ]
)

# Display checkpoint
display(Markdown(f"### 📚 Checkpoint Created\n**Topic:** {checkpoint.topic}"))
display(Markdown("**Objectives:**"))
for i, obj in enumerate(checkpoint.objectives, 1):
    display(Markdown(f"{i}. {obj}"))

print("\n✅ Checkpoint creation successful!")

### Test 2: Initialize State

In [ ]:
# Create initial state with user notes
user_notes = """
Functions in Python are defined using the 'def' keyword.
They can take parameters and return values.
Example: def greet(name): return f"Hello {name}"
"""

state = create_initial_state(
    checkpoint=checkpoint,
    user_notes=user_notes
)

# Display state
display(Markdown("### 🔧 Initial State Created"))
display(Markdown(f"**Current Stage:** {state['current_stage']}"))
display(Markdown(f"**Retry Count:** {state['retry_count']}"))
display(Markdown(f"**Context Valid:** {state['context_valid']}"))

print("\n✅ State initialization successful!")

### Test 3: Context Manager

In [ ]:
# Initialize context manager
context_manager = ContextManager(chunk_size=1000, chunk_overlap=200)

display(Markdown("### 📝 Context Manager Initialized"))
display(Markdown(f"- Has LLM: {hasattr(context_manager, 'llm')}"))
display(Markdown(f"- Has Reasoning LLM: {hasattr(context_manager, 'reasoning_llm')}"))
display(Markdown(f"- Has Text Splitter: {hasattr(context_manager, 'text_splitter')}"))

print("\n✅ Context manager initialization successful!")

## 3️⃣ Run Complete Workflow

In [ ]:
# Create learning graph
graph = create_learning_graph()

display(Markdown("### 📊 Learning Graph Created"))
print("Graph is ready to execute the workflow.")
print("\n✅ Graph creation successful!")

In [ ]:
# Execute the workflow
display(Markdown("### ⚙️ Executing Workflow..."))
display(Markdown("*This may take a few moments...*"))

result = graph.invoke(state)

display(Markdown("\n### ✅ Workflow Completed!"))

## 4️⃣ Analyze Results

In [ ]:
# Display workflow results
display(Markdown("### 📊 Workflow Results Summary"))

# Create results table
results_html = f"""
<table style="width:100%; border-collapse: collapse; margin: 20px 0;">
    <tr style="background-color: #f0f0f0;">
        <th style="border: 1px solid #ddd; padding: 12px; text-align: left;">Metric</th>
        <th style="border: 1px solid #ddd; padding: 12px; text-align: left;">Value</th>
    </tr>
    <tr>
        <td style="border: 1px solid #ddd; padding: 12px;">Current Stage</td>
        <td style="border: 1px solid #ddd; padding: 12px; font-weight: bold;">{result['current_stage']}</td>
    </tr>
    <tr style="background-color: #f9f9f9;">
        <td style="border: 1px solid #ddd; padding: 12px;">Contexts Gathered</td>
        <td style="border: 1px solid #ddd; padding: 12px; font-weight: bold;">{len(result['gathered_contexts'])}</td>
    </tr>
    <tr>
        <td style="border: 1px solid #ddd; padding: 12px;">Context Valid</td>
        <td style="border: 1px solid #ddd; padding: 12px; font-weight: bold; color: {'green' if result.get('context_valid') else 'red'};">
            {'✅ Valid' if result.get('context_valid') else '❌ Invalid'}
        </td>
    </tr>
    <tr style="background-color: #f9f9f9;">
        <td style="border: 1px solid #ddd; padding: 12px;">Retry Count</td>
        <td style="border: 1px solid #ddd; padding: 12px; font-weight: bold;">{result['retry_count']}</td>
    </tr>
</table>
"""

display(HTML(results_html))

# Display error if any
if result.get('error'):
    display(Markdown(f"### ⚠️ Error Occurred"))
    display(Markdown(f"**Error:** {result['error']}"))
    if "API key" in result['error'] or "GITHUB_TOKEN" in result['error']:
        display(Markdown("*💡 Note: Configure your .env file with API keys to enable full functionality.*"))

### Detailed Context Analysis

In [ ]:
# Analyze gathered contexts
contexts = result['gathered_contexts']

if contexts:
    display(Markdown(f"### 📄 Gathered Contexts ({len(contexts)} total)"))
    
    # Statistics
    user_notes_count = sum(1 for c in contexts if c.source == "user_notes")
    web_count = sum(1 for c in contexts if c.source != "user_notes")
    avg_relevance = sum(c.relevance_score for c in contexts) / len(contexts)
    
    stats_html = f"""
    <div style="display: flex; gap: 20px; margin: 20px 0;">
        <div style="flex: 1; padding: 15px; background-color: #e3f2fd; border-radius: 8px; text-align: center;">
            <h3 style="margin: 0; color: #1976d2;">📝 User Notes</h3>
            <p style="font-size: 2em; margin: 10px 0; font-weight: bold;">{user_notes_count}</p>
        </div>
        <div style="flex: 1; padding: 15px; background-color: #f3e5f5; border-radius: 8px; text-align: center;">
            <h3 style="margin: 0; color: #7b1fa2;">🌐 Web Search</h3>
            <p style="font-size: 2em; margin: 10px 0; font-weight: bold;">{web_count}</p>
        </div>
        <div style="flex: 1; padding: 15px; background-color: #e8f5e9; border-radius: 8px; text-align: center;">
            <h3 style="margin: 0; color: #388e3c;">⭐ Avg Relevance</h3>
            <p style="font-size: 2em; margin: 10px 0; font-weight: bold;">{avg_relevance:.2f}</p>
        </div>
    </div>
    """
    display(HTML(stats_html))
    
    # Show each context
    for i, context in enumerate(contexts, 1):
        display(Markdown(f"#### Context {i}: {context.source}"))
        display(Markdown(f"**Relevance Score:** {context.relevance_score:.2f}"))
        display(Markdown(f"**Chunk ID:** {context.chunk_id}"))
        display(Markdown("**Content:**"))
        
        # Display content in a box
        content_html = f"""
        <div style="padding: 15px; background-color: #f5f5f5; border-left: 4px solid #2196f3; 
                    border-radius: 4px; margin: 10px 0; font-family: monospace; white-space: pre-wrap;">
            {context.content[:500]}{'...' if len(context.content) > 500 else ''}
        </div>
        """
        display(HTML(content_html))
        print()  # Add spacing
else:
    display(Markdown("### ℹ️ No contexts gathered"))
    display(Markdown("This may be due to missing API configuration or an error in the workflow."))

### Validation Message Analysis

In [ ]:
# Display validation message
if result.get('validation_message'):
    display(Markdown("### 🔍 Validation Details"))
    
    validation_html = f"""
    <div style="padding: 20px; background-color: #fff3e0; border-left: 4px solid #ff9800; 
                border-radius: 4px; margin: 10px 0;">
        <p style="margin: 0; white-space: pre-wrap;">{result['validation_message']}</p>
    </div>
    """
    display(HTML(validation_html))
else:
    display(Markdown("*No validation message available*"))

## 5️⃣ Export Results

In [ ]:
# Export results to JSON for further analysis
export_data = {
    'timestamp': str(checkpoint.timestamp),
    'topic': checkpoint.topic,
    'objectives': checkpoint.objectives,
    'current_stage': result['current_stage'],
    'contexts_count': len(result['gathered_contexts']),
    'context_valid': result.get('context_valid', False),
    'retry_count': result['retry_count'],
    'error': result.get('error'),
    'contexts': [
        {
            'source': c.source,
            'relevance_score': c.relevance_score,
            'content_length': len(c.content),
            'chunk_id': c.chunk_id
        }
        for c in result['gathered_contexts']
    ]
}

# Save to file
output_file = Path('workflow_results.json')
with open(output_file, 'w') as f:
    json.dump(export_data, f, indent=2)

display(Markdown(f"### 💾 Results Exported"))
display(Markdown(f"Results saved to: `{output_file}`"))
print("\n✅ Export successful!")

## 6️⃣ Summary & Next Steps

In [ ]:
# Generate summary
summary_html = f"""
<div style="padding: 30px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
            color: white; border-radius: 12px; margin: 20px 0;">
    <h2 style="margin-top: 0; color: white;">📊 Workflow Summary</h2>
    <ul style="font-size: 1.1em; line-height: 1.8;">
        <li>✅ <strong>Topic:</strong> {checkpoint.topic}</li>
        <li>✅ <strong>Objectives:</strong> {len(checkpoint.objectives)} learning goals</li>
        <li>✅ <strong>Final Stage:</strong> {result['current_stage']}</li>
        <li>✅ <strong>Contexts:</strong> {len(result['gathered_contexts'])} gathered</li>
        <li>✅ <strong>Status:</strong> {'Completed successfully' if not result.get('error') else 'Completed with warnings'}</li>
    </ul>
</div>

<div style="padding: 20px; background-color: #e3f2fd; border-radius: 8px; margin: 20px 0;">
    <h3 style="margin-top: 0; color: #1976d2;">🚀 Next Steps</h3>
    <ol style="line-height: 1.8;">
        <li>Review the gathered contexts above</li>
        <li>Check the validation results</li>
        <li>Export results using the cell above</li>
        <li>Try with different topics and objectives</li>
        <li>Use the Streamlit dashboard for real-time visualization</li>
    </ol>
</div>
"""

display(HTML(summary_html))

---

## 🎓 Analysis Complete!

This notebook demonstrated:
- ✅ Component testing (Checkpoint, State, Context Manager)
- ✅ Complete workflow execution
- ✅ Real-time result analysis
- ✅ Data export for further analysis

**For real-time visualization, run:**
```bash
streamlit run streamlit_app.py
```